Add your import statements and the database connection statements in the below code block

Database file path = '/course/data/CSE-578/dinofunworld.db'

In [1]:
# your code here
import sqlite3
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
db_filename = '/course/data/CSE-578/dinofunworld.db'
conn = sqlite3.connect(db_filename)
c = conn.cursor()

### **Question 1:**
Create a distance matrix suitable for use in hierarchical clustering of the checkin sequences of the 5 specified visitors.

Your distance function should count the number of dissimilarities in the sequences without considering any other factors.

**Note** 
* The distance matrix should be reported as a dictionary of dictionaries (eg. {1: {2:0, 3:0, 4:0}, 2: {1:0, 3:0, ...}, ...}).
* The order of the output is not important.

In [42]:
### TEST FUNCTION: test_question1
# DO NOT REMOVE OR MODIFY THE ABOVE LINE
# your code here
fiveVisitorsID = ['165316', '1835254', '296394', '404385', '448990']
sequences = {}
for visitorID in fiveVisitorsID:
    query = "SELECT sequence FROM sequences  WHERE visitorID=" + visitorID
    #print(query)
    c.execute(query)
    result = c.fetchone()
    #print(result)
    if result:
        tmpresult = result[0].split('-')
        sequences[visitorID] = tmpresult

#calculate dissimilarity/distance FUNCTION (1 dissimilar = distance + 1)
def cal_dist(seqX, seqY):
    lenX = len(seqX) #X lenth will equal Y length from discription
    dissimilar_count = 0
    for i in range(lenX):
        if seqX[i] != seqY[i]:
            dissimilar_count += 1
    return dissimilar_count

#distance matrix init
distance_matrix = {int(visitorID): {} for visitorID in fiveVisitorsID}
#print(distance_matrix)
for i, visitorX in enumerate(fiveVisitorsID):
    for j, visitorY in enumerate(fiveVisitorsID):
        if visitorX != visitorY:
            dist = cal_dist(sequences[visitorX], sequences[visitorY])
            tmpvisitorX = int(visitorX)
            tmpvisitorY = int(visitorY)
            distance_matrix[tmpvisitorX][tmpvisitorY] = dist

print(distance_matrix)

{165316: {1835254: 336, 296394: 318, 404385: 329, 448990: 318}, 1835254: {165316: 336, 296394: 282, 404385: 298, 448990: 282}, 296394: {165316: 318, 1835254: 282, 404385: 303, 448990: 0}, 404385: {165316: 329, 1835254: 298, 296394: 303, 448990: 303}, 448990: {165316: 318, 1835254: 282, 296394: 0, 404385: 303}}


### **Question 2:**  
Create and display a Parallel Coordinate Plot displaying the minimum, maximum and average attendance for each ride in the park.

For this question, display a Parallel Coordinate Plot in the notebook and print the data used to create a Parallel Coordinate Plot as a dictionary of dictionaries (eg: {  'Ride1' : {min : 1, max : 3, avg : 2 }, 'Ride2' :{ min : 1, max : 3, avg : 2 } ... })

**Note:** 
* Not all attractions are rides.
* The output should be a dictionary and the order of the output is not important.
* Use the parallel coordinates function from the **pandas library** to generate the graph.
* The Parallel Coordinate plot must have the below mentioned parameters,
    * legend and it should be outside the plot.
    * y-label ('**Attendance**')
    * title named '**Ride Attendance - Minimum, Maximum, Average**'
* The plot must not have the following parameters,
    * do not set figure size
    * do not set font size or font-weight for titles and labels

In [0]:
### TEST FUNCTION: test_question2
# DO NOT REMOVE OR MODIFY THE ABOVE LINE
# your code here


### **Question 3:**
Create and display a Scatterplot Matrix displaying the minimum, maximum and average attendance for each ride in the park.


**Note:** 
* Print the output values of Question 2 as the same data will be used for Scatterplot.
* The output should be a dictionary (eg: {  'Ride1' : {min : 1, max : 3, avg : 2 }, 'Ride2' :{ min : 1, max : 3, avg : 2 } ... }) and the order of the output is not important.
* This is a different view into the same data as the previous part. Use the scatter plot matrix function from the **pandas library** to generate the graph.
* While you work on these plots, consider the different things that each chart says about the data.
* The Scatterplot must have the below mentioned parameters,
    * title named ' **Scatterplot Matrix - Minimum, Maximum, Average Attendance** '
* The plot must not have the following parameters,
    * legend
    * do not set figure size
    * do not set font size or font-weight for titles and labels

In [0]:
### TEST FUNCTION: test_question3
# DO NOT REMOVE OR MODIFY THE ABOVE LINE
# your code here
